In [4]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import keras_tuner as kt
import os

X_train, y_train, X_val, y_val = None, None, None, None

def create_dataset(X, y, time_steps=1):

    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)] 
        Xs.append(v)
        ys.append(y[i + time_steps]) 
    return np.array(Xs), np.array(ys)

def build_model(hp):

    model = Sequential()

    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)

    model.add(LSTM(units=hp_units, return_sequences=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))

    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(LSTM(units=hp_units, return_sequences=False))

    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(units=25))
    model.add(Dense(units=1))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mean_squared_error')

    return model

def train_lstm_model():

    global X_train, y_train, X_val, y_val

    master_dataset_path = 'reliance_master_dataset.csv' 
    output_model_path = 'reliance_lstm_model_tuned.keras' #
    time_steps = 60 
    print("\nStarting LSTM Model Tuning and Training for Reliance Industries ")
    try:
        print(f"Loading data from {master_dataset_path}...")
        if not os.path.exists(master_dataset_path):
            raise FileNotFoundError(f"Master dataset not found: {master_dataset_path}")

        df = pd.read_csv(master_dataset_path)
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        features = df.select_dtypes(include=np.number)
        target = features['close'].values.reshape(-1, 1)

        print("Scaling features...")
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_features = scaler.fit_transform(features)

        target_scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_target = target_scaler.fit_transform(target)

        print(f"Creating sequences with {time_steps} time steps...")
        X, y = create_dataset(scaled_features, scaled_target, time_steps)

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
        print(f"Training data shape: {X_train.shape}")
        print(f"Validation data shape: {X_val.shape}")

        print("\nSetting up KerasTuner for hyperparameter search...")

        tuner = kt.RandomSearch(
            build_model,
            objective='val_loss',
            max_trials=10,
            executions_per_trial=1,
            directory='reliance_lstm_tuner', 
            project_name='reliance_price_prediction'
        )

        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

        print("\nStarting tuner search... (This will take a significant amount of time)")

        tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[stop_early])

        print("\nTuner search complete. Retrieving best model...")
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        print(f"""
        Best model hyperparameters:
        - Units: {best_hps.get('units')}
        - Dropout 1: {best_hps.get('dropout_1'):.2f}
        - Dropout 2: {best_hps.get('dropout_2'):.2f}
        - Learning Rate: {best_hps.get('learning_rate')}
        """)

        model = tuner.hypermodel.build(best_hps)

        print("\nRetraining the best model on the full training data...")
        history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[stop_early])

        print(f"\nSaving the tuned LSTM model to {output_model_path}...")
        model.save(output_model_path)
        print("Model saved successfully.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure the master dataset was created successfully.")
    except Exception as e:
        print(f"An error occurred during LSTM model training: {e}")

    print("\n--- LSTM Model Training Completed ---")

if __name__ == '__main__':
    train_lstm_model()


Trial 10 Complete [00h 00m 38s]
val_loss: 0.02987702563405037

Best val_loss So Far: 0.009773152880370617
Total elapsed time: 00h 04m 20s

Tuner search complete. Retrieving best model...

        Best model hyperparameters:
        - Units: 128
        - Dropout 1: 0.30
        - Dropout 2: 0.10
        - Learning Rate: 0.001
        

Retraining the best model on the full training data...
Epoch 1/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0443 - val_loss: 0.0728
Epoch 2/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0014 - val_loss: 0.0354
Epoch 3/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 8.3049e-04 - val_loss: 0.0223
Epoch 4/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 6.6959e-04 - val_loss: 0.0195
Epoch 5/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 5.1652e-04 - val_loss: 0.0235
Epoch 6/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 4.2146e-04 - val_loss: 0.0254
Epoch 7/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.8989e-